Para a construção de um modelo utilizando o Spark e MLlib são necessários 6 passos:

#1) Construir e iniciar a seção SPARK
#2) Implementar o carregamento dos dados para o spark: Carregar o arquivo, especificar o formato desejado e lê os dados como um Dataframe do Spark
#3) Identificar as características a serem utilizadas para treinamento e teste do modelo
#4) Instanciar as classes e os objetos dos algoritmos a serem utilizados
#5) Utilizar o método fit() para realizar o treinamento do modelo
#5) Avaliar o modelo

In [2]:
from pyspark.ml.classification import LogisticRegression as LR  # utilizada para importar a classificação através da regressão logística
from pyspark.ml.feature import VectorAssembler #Utilizada para construir o vetor/matriz que vamos utilizar para a classificação
from pyspark.ml.feature import StringIndexer #Atribui a cada valor literal um valor inteiro para utilizar no algoritmo 
from pyspark.ml import Pipeline # Utilizada para criar o pipeline de utilização dos dados

In [3]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [4]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("ClassificadorVinhos").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [5]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/d1995_07_01-24d0c.json,d1995_07_01-24d0c.json,7451741
dbfs:/FileStore/tables/d1995_07_02-c3f44.json,d1995_07_02-c3f44.json,9572086
dbfs:/FileStore/tables/d1995_07_03-f99af.json,d1995_07_03-f99af.json,14125303
dbfs:/FileStore/tables/d1995_07_04-c5a7f.json,d1995_07_04-c5a7f.json,12030333
dbfs:/FileStore/tables/d1995_07_05-0f261.json,d1995_07_05-0f261.json,14662194
dbfs:/FileStore/tables/d1995_07_06-20619.json,d1995_07_06-20619.json,15557682
dbfs:/FileStore/tables/d1995_07_07-2dd8d.json,d1995_07_07-2dd8d.json,15279295
dbfs:/FileStore/tables/d1995_07_08-83302.json,d1995_07_08-83302.json,7033852
dbfs:/FileStore/tables/d1995_07_09-f75d3.json,d1995_07_09-f75d3.json,5589076
dbfs:/FileStore/tables/d1995_07_10-160a3.json,d1995_07_10-160a3.json,10616482


In [6]:
diretorioDataset="/FileStore/tables/winequality_red-42ff5.csv"

In [7]:
#realiza a carga do dataset
wine_df = spark.read.format("csv"). \
    options(header="true",\
    inferschema="true",sep=';'). \
    load(diretorioDataset)

In [8]:
#realiza o print do esquema do dataframe
wine_df.printSchema()

root
-- fixed acidity: double (nullable = true)
-- volatile acidity: double (nullable = true)
-- citric acid: double (nullable = true)
-- residual sugar: double (nullable = true)
-- chlorides: double (nullable = true)
-- free sulfur dioxide: double (nullable = true)
-- total sulfur dioxide: double (nullable = true)
-- density: double (nullable = true)
-- pH: double (nullable = true)
-- sulphates: double (nullable = true)
-- alcohol: double (nullable = true)
-- quality: integer (nullable = true)

In [9]:
#realiza o print do dataset
wine_df.show(15)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5|
 7.8| 0.88| 0.0| 2.6| 0.098| 25.0| 67.0| 0.9968| 3.2| 0.68| 9.8| 5|
 7.8| 0.76| 0.04| 2.3| 0.092| 15.0| 54.0| 0.997|3.26| 0.65| 9.8| 5|
 11.2| 0.28| 0.56| 1.9| 0.075| 17.0| 60.0| 0.998|3.16| 0.58| 9.8| 6|
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5|
 7.4| 0.66| 0.0| 1.8| 0.075| 13.0| 40.0| 0.9978|3.51| 0.56| 9.4| 5|
 7.9| 0.6| 0.06| 1.6| 0.069| 15.0| 59.0| 0.9964| 3.3| 0.46| 9.4| 5|
 7.3| 0.65| 0.0| 1.2| 0.065| 15.0| 21.0| 0.9946|3.39| 0.47| 10.0| 7|
 7.8| 0.58| 0.02| 2.0| 0.073| 9.0| 18.0| 0.9968|3.36| 0.57| 9.5| 7|
 7.5| 0.5| 0.36| 6.1| 0.071| 17.0| 102.0| 0.9978|3.35| 0.8| 10.5| 5|
 6.7| 0.58| 0.08| 1.8| 0.097| 15.0| 65.0| 0.9959|3.28| 0.54| 9.2| 5|
 7.5| 0.5| 0.36| 6.1| 0.071| 17.0| 102.0| 0.9978|3.35| 0.8| 10.5| 5|
 5.6| 0.615| 0.0| 1.6| 0.089| 16.0| 59.0| 0.9943|3.58| 0.52| 9.9| 5|
 7.8| 0.61| 0.29| 1.6| 0.114| 9.0| 29.0| 0.9974|3.26| 1.56| 9.1| 5|
 8.9| 0.62| 0.18| 3.8| 0.176| 52.0| 145.0| 0.9986|3.16| 0.88| 9.2| 5|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
only showing top 15 rows

In [10]:
#realizando um filtro para o dataframe
wine_df.filter(wine_df.quality>7).show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
 7.9| 0.35| 0.46| 3.6| 0.078| 15.0| 37.0| 0.9973|3.35| 0.86| 12.8| 8|
 10.3| 0.32| 0.45| 6.4| 0.073| 5.0| 13.0| 0.9976|3.23| 0.82| 12.6| 8|
 5.6| 0.85| 0.05| 1.4| 0.045| 12.0| 88.0| 0.9924|3.56| 0.82| 12.9| 8|
 12.6| 0.31| 0.72| 2.2| 0.072| 6.0| 29.0| 0.9987|2.88| 0.82| 9.8| 8|
 11.3| 0.62| 0.67| 5.2| 0.086| 6.0| 19.0| 0.9988|3.22| 0.69| 13.4| 8|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
only showing top 5 rows

In [11]:
#realizando um filtro para o dataframe
wine_df.filter(wine_df.quality>8).show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+-----------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|quality_new|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+-----------+
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+-----------+

In [12]:
#importa a função para criar uma nova coluna no dataset (similar às funções do SQL)
from pyspark.sql.functions import when


In [13]:
#cria a nova coluna que contém a classificação como ruim, boa ou mediana
wine_df = wine_df.withColumn('quality_new',\
    when(wine_df['quality']< 5, 0 ).\
    otherwise(when(wine_df['quality']<8,1)\
    .otherwise(2)))

In [14]:
#realiza o print do dataset
wine_df.show(15)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|quality_new|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5| 1|
 7.8| 0.88| 0.0| 2.6| 0.098| 25.0| 67.0| 0.9968| 3.2| 0.68| 9.8| 5| 1|
 7.8| 0.76| 0.04| 2.3| 0.092| 15.0| 54.0| 0.997|3.26| 0.65| 9.8| 5| 1|
 11.2| 0.28| 0.56| 1.9| 0.075| 17.0| 60.0| 0.998|3.16| 0.58| 9.8| 6| 1|
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5| 1|
 7.4| 0.66| 0.0| 1.8| 0.075| 13.0| 40.0| 0.9978|3.51| 0.56| 9.4| 5| 1|
 7.9| 0.6| 0.06| 1.6| 0.069| 15.0| 59.0| 0.9964| 3.3| 0.46| 9.4| 5| 1|
 7.3| 0.65| 0.0| 1.2| 0.065| 15.0| 21.0| 0.9946|3.39| 0.47| 10.0| 7| 1|
 7.8| 0.58| 0.02| 2.0| 0.073| 9.0| 18.0| 0.9968|3.36| 0.57| 9.5| 7| 1|
 7.5| 0.5| 0.36| 6.1| 0.071| 17.0| 102.0| 0.9978|3.35| 0.8| 10.5| 5| 1|
 6.7| 0.58| 0.08| 1.8| 0.097| 15.0| 65.0| 0.9959|3.28| 0.54| 9.2| 5| 1|
 7.5| 0.5| 0.36| 6.1| 0.071| 17.0| 102.0| 0.9978|3.35| 0.8| 10.5| 5| 1|
 5.6| 0.615| 0.0| 1.6| 0.089| 16.0| 59.0| 0.9943|3.58| 0.52| 9.9| 5| 1|
 7.8| 0.61| 0.29| 1.6| 0.114| 9.0| 29.0| 0.9974|3.26| 1.56| 9.1| 5| 1|
 8.9| 0.62| 0.18| 3.8| 0.176| 52.0| 145.0| 0.9986|3.16| 0.88| 9.2| 5| 1|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+
only showing top 15 rows

In [15]:
#realiza o print do dataset
wine_df.filter(wine_df.quality_new>1).show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|quality_new|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+
 7.9| 0.35| 0.46| 3.6| 0.078| 15.0| 37.0| 0.9973|3.35| 0.86| 12.8| 8| 2|
 10.3| 0.32| 0.45| 6.4| 0.073| 5.0| 13.0| 0.9976|3.23| 0.82| 12.6| 8| 2|
 5.6| 0.85| 0.05| 1.4| 0.045| 12.0| 88.0| 0.9924|3.56| 0.82| 12.9| 8| 2|
 12.6| 0.31| 0.72| 2.2| 0.072| 6.0| 29.0| 0.9987|2.88| 0.82| 9.8| 8| 2|
 11.3| 0.62| 0.67| 5.2| 0.086| 6.0| 19.0| 0.9988|3.22| 0.69| 13.4| 8| 2|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+
only showing top 5 rows

In [16]:
#converte os valores para string - não executa, é apenas transformação
string_index = StringIndexer(inputCol='quality_new',\
    outputCol='quality'+'Index')

In [17]:
#Seleciona as colunas a serem utilizadas como entradas para a classificação - não executa, é apenas transformação
vectors = VectorAssembler(inputCols = \
    ['fixed acidity','volatile acidity',\
    'citric acid','residual sugar','chlorides',\
    'free sulfur dioxide', 'total sulfur dioxide', \
    'density','pH','sulphates', 'alcohol'],\
    outputCol = 'features')

In [18]:
#cria o "novo dataframe" - não executa, é apenas transformação
stages = [vectors, string_index]

In [19]:
#define o pipeline
pipeline = Pipeline().setStages(stages)

In [20]:
#aplicando as transformações ao dataset - executa o comando/ação
pipelineModel = pipeline.fit(wine_df)

In [21]:
#aplica as transformações obtidas através do pipeline
pl_data_df = pipelineModel.transform(wine_df)

In [22]:
#print do dataset após a aplicação do pipeline
pl_data_df.show(15)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|quality_new| features|qualityIndex|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5| 1|[7.4,0.7,0.0,1.9,...| 0.0|
 7.8| 0.88| 0.0| 2.6| 0.098| 25.0| 67.0| 0.9968| 3.2| 0.68| 9.8| 5| 1|[7.8,0.88,0.0,2.6...| 0.0|
 7.8| 0.76| 0.04| 2.3| 0.092| 15.0| 54.0| 0.997|3.26| 0.65| 9.8| 5| 1|[7.8,0.76,0.04,2....| 0.0|
 11.2| 0.28| 0.56| 1.9| 0.075| 17.0| 60.0| 0.998|3.16| 0.58| 9.8| 6| 1|[11.2,0.28,0.56,1...| 0.0|
 7.4| 0.7| 0.0| 1.9| 0.076| 11.0| 34.0| 0.9978|3.51| 0.56| 9.4| 5| 1|[7.4,0.7,0.0,1.9,...| 0.0|
 7.4| 0.66| 0.0| 1.8| 0.075| 13.0| 40.0| 0.9978|3.51| 0.56| 9.4| 5| 1|[7.4,0.66,0.0,1.8...| 0.0|
 7.9| 0.6| 0.06| 1.6| 0.069| 15.0| 59.0| 0.9964| 3.3| 0.46| 9.4| 5| 1|[7.9,0.6,0.06,1.6...| 0.0|
 7.3| 0.65| 0.0| 1.2| 0.065| 15.0| 21.0| 0.9946|3.39| 0.47| 10.0| 7| 1|[7.3,0.65,0.0,1.2...| 0.0|
 7.8| 0.58| 0.02| 2.0| 0.073| 9.0| 18.0| 0.9968|3.36| 0.57| 9.5| 7| 1|[7.8,0.58,0.02,2....| 0.0|
 7.5| 0.5| 0.36| 6.1| 0.071| 17.0| 102.0| 0.9978|3.35| 0.8| 10.5| 5| 1|[7.5,0.5,0.36,6.1...| 0.0|
 6.7| 0.58| 0.08| 1.8| 0.097| 15.0| 65.0| 0.9959|3.28| 0.54| 9.2| 5| 1|[6.7,0.58,0.08,1....| 0.0|
 7.5| 0.5| 0.36| 6.1| 0.071| 17.0| 102.0| 0.9978|3.35| 0.8| 10.5| 5| 1|[7.5,0.5,0.36,6.1...| 0.0|
 5.6| 0.615| 0.0| 1.6| 0.089| 16.0| 59.0| 0.9943|3.58| 0.52| 9.9| 5| 1|[5.6,0.615,0.0,1....| 0.0|
 7.8| 0.61| 0.29| 1.6| 0.114| 9.0| 29.0| 0.9974|3.26| 1.56| 9.1| 5| 1|[7.8,0.61,0.29,1....| 0.0|
 8.9| 0.62| 0.18| 3.8| 0.176| 52.0| 145.0| 0.9986|3.16| 0.88| 9.2| 5| 1|[8.9,0.62,0.18,3....| 0.0|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
only showing top 15 rows

In [23]:
#divide o dataset entre treinamento e teste (70% treinamento e 30% teste)
train_df, test_df = pl_data_df.randomSplit([0.7,0.3])

In [24]:
train_df.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|quality_new| features|qualityIndex|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
 4.6| 0.52| 0.15| 2.1| 0.054| 8.0| 65.0| 0.9934| 3.9| 0.56| 13.1| 4| 0|[4.6,0.52,0.15,2....| 1.0|
 4.7| 0.6| 0.17| 2.3| 0.058| 17.0| 106.0| 0.9932|3.85| 0.6| 12.9| 6| 1|[4.7,0.6,0.17,2.3...| 0.0|
 5.0| 0.38| 0.01| 1.6| 0.048| 26.0| 60.0|0.99084| 3.7| 0.75| 14.0| 6| 1|[5.0,0.38,0.01,1....| 0.0|
 5.0| 0.4| 0.5| 4.3| 0.046| 29.0| 80.0| 0.9902|3.49| 0.66| 13.6| 6| 1|[5.0,0.4,0.5,4.3,...| 0.0|
 5.0| 0.42| 0.24| 2.0| 0.06| 19.0| 50.0| 0.9917|3.72| 0.74| 14.0| 8| 2|[5.0,0.42,0.24,2....| 2.0|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
only showing top 5 rows

In [25]:
test_df.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|quality_new| features|qualityIndex|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
 4.9| 0.42| 0.0| 2.1| 0.048| 16.0| 42.0|0.99154|3.71| 0.74| 14.0| 7| 1|[4.9,0.42,0.0,2.1...| 0.0|
 5.0| 1.02| 0.04| 1.4| 0.045| 41.0| 85.0| 0.9938|3.75| 0.48| 10.5| 4| 0|[5.0,1.02,0.04,1....| 1.0|
 5.2| 0.48| 0.04| 1.6| 0.054| 19.0| 106.0| 0.9927|3.54| 0.62| 12.2| 7| 1|[5.2,0.48,0.04,1....| 0.0|
 5.2| 0.645| 0.0| 2.15| 0.08| 15.0| 28.0|0.99444|3.78| 0.61| 12.5| 6| 1|[5.2,0.645,0.0,2....| 0.0|
 5.3| 0.47| 0.11| 2.2| 0.048| 16.0| 89.0|0.99182|3.54| 0.88| 13.6| 7| 1|[5.3,0.47,0.11,2....| 0.0|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----------+--------------------+------------+
only showing top 5 rows

In [26]:
#instancia a classe para a execução do modelo através da regressão logística
classificador= LR(featuresCol = 'features', \
    labelCol='qualityIndex',\
               maxIter=50) #maximo de iterações

In [27]:
#aplica o treinamento do modelo
modelo = classificador.fit(train_df)

In [28]:
#obtém o sumário (dados de análise) para o modelo criado
modelSummary = modelo.summary

In [29]:
    # print das estatísticas do modelo
    accuracy = modelSummary.accuracy #acurácia da classificação
    fPR = modelSummary.weightedFalsePositiveRate #taxa de falsos positivos
    tPR = modelSummary.weightedTruePositiveRate #taxa de verdadeiros positivos
    fMeasure = modelSummary.weightedFMeasure() #f-score
    precision = modelSummary.weightedPrecision #precision
    recall = modelSummary.weightedRecall #recall
    print("Acurácia: {}\
        Taxa de verdadeiros positivos {} F-score {} Precision {} Recall {}"\
        .format(accuracy, tPR, fMeasure, precision, recall))

Acurácia: 0.9518486672398968 Taxa de verdadeiros positivos 0.9518486672398968 F-score 0.9314897179054992 Precision 0.9272766794988118 Recall 0.9518486672398968